---
# Objetivo: Treinar modelo BERT no conjunto de dados filtrado pela categoria Grãos. Variavel target o tema dessa categoria. Ex. Arroz, Trigo, Soja, Feijão
---

**1- Descrição do problema ou tarefa:**

Prever o tema da categoria Grãos a partir das perguntas.

**2- Descrição da solução de IA:**
Treinamento supervisionado de modelo de classificação os temas da categoria Grãos (10 classes) com os dados de 5532 perguntas.

**3- Fonte de dados:**


Livro de perguntas e respostas da Embrapa
https://mais500p500r.sct.embrapa.br/view/index.php4

**4-Variáveis independentes:** 
perguntas

**5- Variável dependente:** 
Temas da cagetoria frutas:Arroz, Trigo, Soja, Feijão...

-----------------------------------------------------------------
Autor do caderno: Wellington Rangel
Data: 23/11/2021

In [ ]:
# install ktrain on Google Colab
!pip3 install ktrain

     |████████████████████████████████| 25.3 MB 1.9 MB/s 
     |████████████████████████████████| 6.8 MB 22.2 MB/s 
     |████████████████████████████████| 981 kB 50.8 MB/s 
     |████████████████████████████████| 263 kB 45.3 MB/s 
     |████████████████████████████████| 2.8 MB 37.4 MB/s 
     |████████████████████████████████| 1.2 MB 36.5 MB/s 
     |████████████████████████████████| 468 kB 44.5 MB/s 
     |████████████████████████████████| 61 kB 461 kB/s 
     |████████████████████████████████| 596 kB 34.4 MB/s 
     |████████████████████████████████| 895 kB 39.5 MB/s 
     |████████████████████████████████| 3.3 MB 36.4 MB/s 
  Created wheel for ktrain: filename=ktrain-0.28.3-py3-none-any.whl size=25292659 sha256=86243b50e29cffb6005783232dee579e7b4710f7e7616ef975ee3f216e48deea
  Stored in directory: /root/.cache/pip/wheels/6a/7e/c3/f46cdfc2b81c54424923b1405d7e670c35cacc11ada9a47b1c
  Created wheel for seqeval: filename=seqeval-0.0.19-py3-none-any.whl size=9929 sha256=a67e0cd8ddb3f50d

In [ ]:
# import ktrain and the ktrain.text modules
import ktrain
from ktrain import text

In [ ]:
ktrain.__version__

'0.28.3'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/projeto

/content/drive/MyDrive/Colab Notebooks/projeto


In [ ]:
import pandas as pd
df = pd.read_csv('dados_agrupados.csv',  sep=',', low_memory=False,encoding='latin-1' )


In [ ]:
df

,Numero,Livro,Capitulo,Pergunta,Resposta,Target,Target_final,Target_final1,Target_final2,Target_final3
0,1,Pera,Generalidades,Qual é o centro de origem da pereira?,São citados três centros de origem da pereira:...,Pera,Frutas,NaN,Frutas,NaN
1,2,Pera,Generalidades,Qual é o centro de origem mais importante?,O centro do Oriente Médio é considerado de imp...,Pera,Frutas,NaN,Frutas,NaN
2,3,Pera,Generalidades,Como ocorreu a disseminação da pereira pelo mu...,"Com base em estudos bioquímicos, verificou-se ...",Pera,Frutas,NaN,Frutas,NaN
3,4,Pera,Generalidades,Quais são as espécies de pereira mais cultivad...,"Na Europa, na América do Norte, na América do ...",Pera,Frutas,NaN,Frutas,NaN
4,5,Pera,Generalidades,Quando a pereira foi introduzida no Brasil?,Não há relatos na literatura sobre a introduçã...,Pera,Frutas,NaN,Frutas,NaN
...,...,...,...,...,...,...,...,...,...,...
16591,497,Uva,NaN,Qual é a produção nacional e a mundial de vinhos?,O Brasil produz cerca de 300 milhões de litros...,Uva,Frutas,NaN,Frutas,NaN
16592,498,Uva,NaN,Quais são os maiores produtores e os maiores e...,"Os maiores produtores são a França, a Espanha ...",Uva,Frutas,NaN,Frutas,NaN
16593,499,Uva,NaN,Que quantidade de vinho o Brasil importa anual...,"Mais de 55 milhões de litros de vinho fino, o ...",Uva,Frutas,NaN,Frutas,NaN
16594,500,Uva,NaN,Qual é o consumo per capita dos países que mai...,"França e Luxemburgo, 55 litros; Portugal, 48 l...",Uva,Frutas,NaN,Frutas,NaN


In [ ]:
df = df.loc[df['Target_final'] == 'Graos']

In [ ]:

df = df[['Pergunta', 'Target']]


In [ ]:
df.head()

,Pergunta,Target
503,Qual é a origem do trigo?,Trigo
504,Quando e onde o trigo começou a ser cultivado ...,Trigo
505,Qual é a classificação taxonômica do trigo cul...,Trigo
506,Quais são as principais espécies de trigo exis...,Trigo
507,O que é melhoramento genético do trigo?,Trigo


In [ ]:
df.tail()

,Pergunta,Target
15587,Quais são os fatores que afetam o preço do sorgo?,Sorgo
15588,Qual é a rentabilidade do sorgo em comparação ...,Sorgo
15589,Qual é a importância econômica da cultura do s...,Sorgo
15590,Mais alguma pergunta?,Sorgo
15591,nosso formulário de atendimento na internet. C...,Sorgo


In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
stopwords = nltk.corpus.stopwords.words('portuguese')


In [ ]:
sub = df[[ 'Pergunta', 'Target']]

In [ ]:
sub['texto_sem_stopwords'] = sub['Pergunta'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stopwords)]))

In [ ]:
sub = sub.dropna()

In [ ]:
sub

,Pergunta,Target,texto_sem_stopwords
503,Qual é a origem do trigo?,Trigo,Qual origem trigo?
504,Quando e onde o trigo começou a ser cultivado ...,Trigo,Quando onde trigo começou ser cultivado Brasil?
505,Qual é a classificação taxonômica do trigo cul...,Trigo,Qual classificação taxonômica trigo cultivado?
506,Quais são as principais espécies de trigo exis...,Trigo,Quais principais espécies trigo existentes mundo?
507,O que é melhoramento genético do trigo?,Trigo,O melhoramento genético trigo?
...,...,...,...
15587,Quais são os fatores que afetam o preço do sorgo?,Sorgo,Quais fatores afetam preço sorgo?
15588,Qual é a rentabilidade do sorgo em comparação ...,Sorgo,Qual rentabilidade sorgo comparação milho inve...
15589,Qual é a importância econômica da cultura do s...,Sorgo,Qual importância econômica cultura sorgo Brasil?
15590,Mais alguma pergunta?,Sorgo,Mais alguma pergunta?


In [ ]:
# Separa os dados de treino e teste (proporção 80/20, stratify para classes desbalanceadas)
from sklearn.model_selection import train_test_split
m_X_train, m_X_test, m_y_train, m_y_test = train_test_split(df.Pergunta, df.Target, test_size=0.2, 
                                                    stratify=df.Target,random_state=42)




In [ ]:
type(m_X_train)

pandas.core.series.Series

In [ ]:
m_y_train

9844       Hortas
1857         Soja
9387     Gergelim
3854        Arroz
13295       Milho
           ...   
15225       Sorgo
9732       Hortas
13227       Milho
3444     Amendoin
11042    Mandioca
Name: Target, Length: 4425, dtype: object

In [ ]:
x_train = m_X_train.tolist()
y_train = m_y_train.tolist()
x_test = m_X_test.tolist()
y_test = m_y_test.tolist()

In [ ]:
type(x_train)

list

In [ ]:

type(x_train)


list

In [ ]:
classes = df.Target.unique().tolist()

In [ ]:
classes

['Trigo',
 'Soja',
 'Amendoin',
 'Arroz',
 'Feijao',
 'Gergelim',
 'Hortas',
 'Mandioca',
 'Milho',
 'Sorgo']

In [ ]:
(x_train,  y_train), (x_test, y_test), preproc = text.texts_from_array(x_train=x_train, y_train=y_train,
                                                                       x_test=x_test, y_test=y_test,
                                                                       class_names=classes,
                                                                       preprocess_mode='bert',
                                                                       maxlen=350, 
                                                                       max_features=35000)

downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: pt


Is Multi-Label? False
preprocessing test...
language: pt


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:621: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


task: text classification


## Treinando o modelo

In [ ]:
# you can disregard the deprecation warnings arising from using Keras 2.2.4 with TensorFlow 1.14.
model = text.text_classifier('bert', train_data=(x_train, y_train), preproc=preproc)
learner = ktrain.get_learner(model, train_data=(x_train, y_train), batch_size=8)

Is Multi-Label? False
maxlen is 350
done.


In [ ]:
learner.fit_onecycle(2e-5, 4)



begin training using onecycle policy with max lr of 2e-05...
Epoch 1/4
554/554 [==============================] - 788s 1s/step - loss: 1.6927 - accuracy: 0.4246
Epoch 2/4
554/554 [==============================] - 759s 1s/step - loss: 0.8202 - accuracy: 0.7304
Epoch 3/4
554/554 [==============================] - 758s 1s/step - loss: 0.6066 - accuracy: 0.7937
Epoch 4/4
554/554 [==============================] - 756s 1s/step - loss: 0.3169 - accuracy: 0.8969


In [ ]:
learner.validate(val_data=(x_test, y_test), class_names=classes)

              precision    recall  f1-score   support

       Trigo       0.76      0.80      0.78       100
        Soja       0.78      0.72      0.75       101
    Amendoin       0.73      0.76      0.75       202
       Arroz       0.76      0.73      0.74       101
      Feijao       0.75      0.78      0.76       100
    Gergelim       0.72      0.86      0.79       100
      Hortas       0.67      0.63      0.65       101
    Mandioca       0.75      0.73      0.74       101
       Milho       0.79      0.74      0.77       101
       Sorgo       0.76      0.67      0.71       100

    accuracy                           0.75      1107
   macro avg       0.75      0.74      0.74      1107
weighted avg       0.75      0.75      0.74      1107



array([[ 80,   0,   8,   2,   2,   3,   3,   1,   0,   1],
       [  3,  73,   9,   4,   1,   2,   6,   0,   1,   2],
       [  1,  12, 154,   6,   8,   4,   4,   6,   2,   5],
       [  6,   0,   6,  74,   0,   4,   6,   3,   1,   1],
       [  4,   0,   6,   0,  78,   7,   0,   2,   1,   2],
       [  1,   0,   4,   1,   6,  86,   1,   0,   1,   0],
       [  3,   0,   6,   0,   3,   8,  64,   3,  12,   2],
       [  2,   2,   6,   4,   3,   3,   2,  74,   0,   5],
       [  3,   1,   4,   4,   0,   2,   4,   5,  75,   3],
       [  2,   5,   7,   3,   3,   0,   6,   5,   2,  67]])

## usando o modelo treinado.

In [ ]:
predictor = ktrain.get_predictor(learner.model, preproc)

In [ ]:
predictor.get_classes()

['Amendoin',
 'Arroz',
 'Feijao',
 'Gergelim',
 'Hortas',
 'Mandioca',
 'Milho',
 'Soja',
 'Sorgo',
 'Trigo']

In [ ]:
df[0:1]

,Pergunta,Target
503,Qual é a origem do trigo?,Trigo


In [ ]:
predictor.predict(df[0:1]['Pergunta'].to_string())

'Trigo'

In [ ]:
df[0:1]['Pergunta']

503    Qual é a origem do trigo?
Name: Pergunta, dtype: object

In [ ]:
# we can visually verify that our prediction of 'sci.med' for this document is correct
print(df[0:1]['Target'])

503    Trigo
Name: Target, dtype: object


In [ ]:
predictor.save('/content/drive/MyDrive/Colab Notebooks/projeto/modelos/my_predictor_graos')

/usr/local/lib/python3.7/dist-packages/keras/engine/functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
